In [1]:
#hide
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#hide
%load_ext autoreload
%autoreload 2

%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [3]:
__all__=['camel2snake','Callback','CancelTrainException','CancelEpochException', 'CancelBatchException' ,'Learner' ,'param_getter','TrainEvalCallback', 'AvgStats' ,'AvgStatsCallback', 'Recorder', 'ParamScheduler',
         'LR_Find', 'CudaCallback' ,'BatchTransformXCallback', 'view_tfm','ProgressbarCallback' ,'DebugCallback', 'NoneReduce', 'MixUp' ,'Hooks','hook_outputs', 'get_batch' ,'is_lin_layer', 'find_modules','model_summary' ,'cnn_learner' ,
         'SaveModelCallback','LoadModelCallback','master_bar','progress_bar','format_time','Beta','unsqueeze','reduce_loss','annealer','cos_1cycle_anneal',
         'combine_scheds','create_phases','sched_1cycle','sched_lin','sched_cos','sched_exp', 'sched_no',
         'show_results', 'ShowImgsCallback']

In [4]:
#hide
__all__

['camel2snake',
 'Callback',
 'CancelTrainException',
 'CancelEpochException',
 'CancelBatchException',
 'Learner',
 'param_getter',
 'TrainEvalCallback',
 'AvgStats',
 'AvgStatsCallback',
 'Recorder',
 'ParamScheduler',
 'LR_Find',
 'CudaCallback',
 'BatchTransformXCallback',
 'view_tfm',
 'ProgressbarCallback',
 'DebugCallback',
 'NoneReduce',
 'MixUp',
 'Hooks',
 'hook_outputs',
 'get_batch',
 'is_lin_layer',
 'find_modules',
 'model_summary',
 'cnn_learner',
 'SaveModelCallback',
 'LoadModelCallback',
 'master_bar',
 'progress_bar',
 'format_time',
 'Beta',
 'unsqueeze',
 'reduce_loss',
 'annealer',
 'cos_1cycle_anneal',
 'combine_scheds',
 'create_phases',
 'sched_1cycle',
 'sched_lin',
 'sched_cos',
 'sched_exp',
 'sched_no',
 'show_results',
 'ShowImgsCallback']

In [5]:
#hide
# !git clone https://github.com/prajwal-suresh13/dl_lib.git

In [6]:
from dl_lib.core.utils import *
from dl_lib.core.optimizers import *

In [ ]:
from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

# Learner

In [ ]:
import re
_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')

def camel2snake(name):
    s1 = re.sub(_camel_re1,r'\1_\2',name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

In [ ]:
class Callback():
    _order=0
    def set_learner(self, learner): self.learner = learner
    def __getattr__(self, k): return getattr(self.learner, k)

    @property
    def name(self):
        name = re.sub(r'Callback$','', self.__class__.__name__)
        return camel2snake(name or 'callback')

    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

In [ ]:
class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass


In [ ]:
def param_getter(m): return m.parameters()

In [ ]:
class Learner():
    def __init__(self, model, data, loss_func, opt_func=adam_opt, lr=1e-2, splitter=param_getter, cbs=None, cbfuncs = None, plot_hypers=[], metrics={}):
        self.model, self.data, self.loss_func, self.opt_func, self.lr, self.splitter= model, data, loss_func, opt_func, lr, splitter
        self.in_train, self.logger, self.opt = False, print, None
        self.plot_hypers, self.metrics =plot_hypers, metrics

        self.cbs = []
        self.add_cb(TrainEvalCallback())
        self.add_cbs(cbs)
        self.add_cbs(cbf() for cbf in listify(cbfuncs))

    def add_cbs(self, cbs):
        for cb in listify(cbs): self.add_cb(cb)

    def add_cb(self, cb):
        cb.set_learner(self)
        setattr(self, cb.name, cb)
        self.cbs.append(cb)


    def remove_cbs(self, cbs):
        for cb in listify(cbs): self.cbs.remove(cb)

    def one_batch(self, i, xb, yb):
        try:
            self.iter = i
            # pdb.set_trace()
            self.xb, self.yb = xb, yb;                          self('before_batch')
            self.pred = self.model(self.xb) ;                   self('after_pred')
            self.loss = self.loss_func(self.pred, self.yb);     self('after_loss')
            if not self.in_train: return
            self.loss.backward();                                self('after_backward')
            self.opt.step();                                     self('after_step')
            self.opt.zero_grad(); 
        except CancelBatchException:                            self('after_cancel_batch')
        finally:                                                self('after_batch')  

    def all_batches(self):
        self.iters = len(self.dl)
        try:
            for i, (xb,yb) in enumerate(self.dl): self.one_batch(i, xb, yb)
        except CancelEpochException:                            self('after_cancel_epoch')   

    def do_before_fit(self, epochs):
        self.epochs, self.loss = epochs, tensor(0.)
        self('before_fit')

    def do_before_epoch(self, epoch):
        self.epoch, self.dl = epoch, self.data.train_dl
        return self('before_epoch')

    def fit(self, epochs, cbs=None, reset_opt=False):
        self.add_cbs(cbs)

        if reset_opt or not self.opt: self.opt = self.opt_func(self.splitter(self.model), lr = self.lr)

        try:
            self.do_before_fit(epochs)
            for epoch in range(epochs):

                if not self.do_before_epoch(epoch): self.all_batches()

                with torch.no_grad():
                    self.dl = self.data.valid_dl
                    if not self('before_validate'): self.all_batches()
                self('after_epoch') 

        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.remove_cbs(cbs)   

    ALL_CBS={'before_batch', 'after_pred', 'after_loss', 'after_backward', 'after_step', 'after_cancel_batch', 'after_batch',
                 'after_cancel_epoch', 'before_fit', 'before_epoch','before_validate', 'after_epoch', 'after_cancel_train', 'after_fit' }

    def __call__(self, cb_name):
        res = False
        assert cb_name in self.ALL_CBS 
        for cb in sorted(self.cbs, key = lambda x: x._order): res = cb(cb_name) and res
        return res
  

# TrainEvalCallback

In [ ]:
class TrainEvalCallback(Callback):
    def before_fit(self):
        self.learner.n_epochs = 0.
        self.learner.n_iters = 0

    def before_epoch(self):
        self.learner.n_epochs = self.epoch
        self.model.train()
        self.learner.in_train= True

    def after_batch(self):
        if not self.in_train: return
        self.learner.n_epochs +=1./self.iters
        self.learner.n_iters +=1

    def before_validate(self):
        self.model.eval()
        self.learner.in_train = False

    

# AvgStatsCallback

In [ ]:
class AvgStats():
    def __init__(self, metrics , in_train): 
        self.metrics, self.in_train = listify(metrics), in_train

    def accumulate(self, learner):
        batch_size = learner.yb.shape[0]
        self.tot_loss += learner.loss *batch_size
        self.count += batch_size
        for i,m in enumerate(self.metrics):
            self.tot_metrics[i] += m(learner.pred, learner.yb) * batch_size

    def reset(self):
        self.tot_loss, self.count = 0., 0
        self.tot_metrics = [0.] * len(self.metrics)

    @property
    def all_stats(self):
        return [self.tot_loss.item()] + self.tot_metrics

    @property
    def avg_stats(self):
        return [o/self.count for o in self.all_stats]

    def __repr__(self):
        if not self.count: return ""
        return f"{'train'if self.in_train else 'valid'} : {self.avg_stats}"
        




In [ ]:
class AvgStatsCallback(Callback):
    def __init__(self, metrics, valid_stats=True):
        self.train_stats =   AvgStats(metrics, True)
        self.val=valid_stats
        if self.val: self.valid_stats = AvgStats(metrics, False)

    def before_fit(self):
        met_names = ['loss'] +[m.__name__ for m in self.train_stats.metrics]
        names = ['epoch'] + [f'train_{n}' for n in met_names] 
        if self.val:names+= [f'valid_{n}' for n in met_names] 
        names+=[f'{name}' for name in self.learner.metrics.keys()]
        names += ['time']
        self.logger(names)

    def before_epoch(self):
        self.train_stats.reset()
        if self.val:self.valid_stats.reset()
        self.start_time = time.time()

    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats if self.val  else None
        if stats is not None:
            with torch.no_grad():stats.accumulate(self.learner)

    def after_epoch(self):
        stats = [str(self.epoch)]
        stats+= [f'{v:.6f}' for v in self.train_stats.avg_stats]
        if self.val: stats+= [f'{v:.6f}' for v in self.valid_stats.avg_stats]
        for value in self.learner.metrics.values():
            try:
                l = operator.attrgetter(value)(self.learner) 
                stats+=[f'{l:.6f}']
            except AttributeError as e:
                try:
                    l = operator.attrgetter(value)(self.learner.loss_func)
                    stats+=[f'{l:.6f}']
                except AttributeError as e:
                    stats+=['Nil'] 
             

        # stats+= [f'{getattr(self.learner, value):.4f}' for value in self.learner.metrics.values()]
        stats+= [format_time(time.time() - self.start_time)]
        self.logger(stats)


 # Recorder and ParamScheduler

In [ ]:
class Recorder(Callback):
    def before_fit(self): 
        self.losses, self.lrs = [],[[] for _ in self.opt.param_groups]
        self.hyper_values={}
        self.loss_metrics={}
        for h in self.learner.plot_hypers:
            self.hyper_values[h]=[[] for _ in self.opt.param_groups]

        
        self.names = self.learner.metrics.keys()
        for name in self.learner.metrics.values():
            self.loss_metrics[name]=[]



    def after_batch(self):
        if not self.in_train: return
        self.losses.append(self.loss.detach().cpu())
        for pg, lr in zip(self.opt.hypers, self.lrs):lr.append(pg['lr'])

        for hyper, values in self.hyper_values.items():
            for pg,value in zip(self.opt.hypers,values): value.append(pg[hyper])
        
        for metric,value in self.loss_metrics.items():
            try:
                l = operator.attrgetter(metric)(self.learner) or operator.attrgetter(metric)(self.learner.loss_func)
                value.append(torch.round(l,decimals=3).cpu())
            except AttributeError as e:
                value.append(0.)

    def plot_lr(self, pgid=-1): plt.plot(self.lrs[pgid])

    def plot_hypers(self, pgid=-1):
        n = len(self.hyper_values.keys())
        rows,columns = int(math.ceil(n/2)),2
        fig,ax = plt.subplots(rows,columns, figsize=(12,12))
        names=list(self.hyper_values.keys())
        values=self.hyper_values.values()
        
        for value,a,name in zip(values,ax.flat,names):
            a.plot(value[pgid],label=name)

        plt.show()

    def plot_loss(self, skip_last=0):
        l = len(self.losses)-skip_last
        fig,ax = plt.subplots(1,1,figsize=(12,12))
        ax.plot(self.losses[:l], label="Loss")
        
        for name,value in zip(self.names,self.loss_metrics.values()):
                ax.plot(value[:l], label=name)

        plt.show()

    def plot(self, pgid=-1, skip_last=0):
        losses = [o.item() for o in self.losses]
        lrs = self.lrs[pgid]
        n = len(self.losses) - skip_last
        plt.xscale('log')
        plt.plot(lrs[:n], losses[:n])

In [ ]:
class ParamScheduler(Callback):
    _order = 1
    def __init__(self, pname, sched_funcs):
        self.pname = pname
        self.sched_funcs = listify(sched_funcs)
        

    def before_batch(self):
        if not self.in_train: return
        if len(self.sched_funcs) == 1: self.sched_funcs = self.sched_funcs*len(self.opt.param_groups)
        assert len(self.sched_funcs) == len(self.opt.param_groups)
        for f,hyper in zip(self.sched_funcs,self.opt.hypers):
            hyper[self.pname] = f(self.n_epochs/self.epochs)



## Schedulers

In [ ]:
def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner

@annealer
def sched_lin(start, end, pos): return start + pos*(end - start)

@annealer
def sched_cos(start, end, pos): return start + (1+math.cos(math.pi*(1-pos))) *(end-start)/2

@annealer
def sched_exp(start, end, pos): return start * (end/start)**pos

@annealer
def sched_no(start, end, pos): return start

def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]

In [ ]:
def combine_scheds(pcts, scheds):
    assert sum(pcts)==1.
    pcts = tensor([0] + listify(pcts))
    assert torch.all(pcts>=0)
    pcts = torch.cumsum(pcts,0)
    def _inner(pos):
        idx = (pos>=pcts).nonzero().max()
        if idx==2: idx=1
        actual_pos = (pos - pcts[idx])/(pcts[idx+1]-pcts[idx])
        return scheds[idx](actual_pos)
    return _inner

In [ ]:
def create_phases(phases):
    phases = listify(phases)
    return phases + [1-sum(phases)]

In [ ]:
def sched_1cycle(lrs, pct_start =0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5)) for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

#LR Find, CudaCallback, BatchTransformXCallback, ProgressbarCallback, DebugCallback

In [ ]:
class LR_Find(Callback):
    _order=1
    def __init__(self, max_iters = 100, min_lr = 1e-6, max_lr=10):
        self.max_iters, self.min_lr, self.max_lr = max_iters, min_lr, max_lr
        self.best_loss = 1e9

    def before_batch(self):
        if not self.in_train: return
        pos = self.n_iters/self.max_iters
        lr = self.min_lr * (self.max_lr/self.min_lr)**pos
        for pg in self.opt.hypers:pg['lr'] = lr

    def after_step(self):
        if self.n_iters>self.max_iters or self.loss>self.best_loss*10:raise CancelTrainException()
        if self.loss < self.best_loss : self.best_loss = self.loss

In [ ]:
class CudaCallback(Callback):
    _order = -30
    def before_fit(self): self.model.cuda()
    def before_batch(self): self.learner.xb, self.learner.yb = self.learner.xb.cuda(), self.learner.yb.cuda()

In [ ]:
class BatchTransformXCallback(Callback):
    _order=2
    def __init__(self, func): self.func = func
    def before_batch(self): self.learner.xb = self.func(self.xb)

def view_tfm(*size):
    def _inner(x): return x.view(*((-1,)+size))
    return _inner

In [ ]:
class ProgressbarCallback(Callback):
    _order =-100

    def before_fit(self):
        self.mbar = master_bar(range(self.epochs))
        self.mbar.on_iter_begin()
        self.learner.logger = partial(self.mbar.write, table=True)

    def after_fit(self):self.mbar.on_iter_end()

    def set_pb(self):
        self.pb = progress_bar(self.dl, parent = self.mbar)
        self.mbar.update(self.epoch)

    def before_epoch(self): self.set_pb()

    def before_validate(self): self.set_pb()
    
    def after_batch(self): self.pb.update(self.iter)

In [ ]:
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

# MixUP

In [ ]:
from torch.distributions.beta import Beta

def unsqueeze(inp, dims):
    for dim in listify(dims):inp=torch.unsqueeze(inp,dim)
    return inp

class NoneReduce():
    def __init__(self, loss_func): self.loss_func, self.old_red = loss_func, None

    def __enter__(self):
        if hasattr(self.loss_func, 'reduction'):
            self.old_red = getattr(self.loss_func, 'reduction')
            setattr(self.loss_func,'reduction','none')
            return self.loss_func
        else: return partial(self.loss_func, reduction='none')

    def __exit__(self, type, value, traceback):
        # print(f'{type}\n\n,{value},\n\n {traceback}')
        if self.old_red is not None: setattr(self.loss_func, 'reduction', self.old_red)

def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

class MixUp(Callback):
    _order = 90
    def __init__(self, alpha=0.4): self.distrib = Beta(tensor([alpha]), tensor([alpha]))

    def before_fit(self):self.old_loss_func, self.learner.loss_func = self.learner.loss_func, self.loss_func

    def before_batch(self):
        if not self.in_train: return
        λ = self.distrib.sample((self.yb.size(0),)).squeeze().to(self.xb.device)
        λ = torch.stack([λ, 1-λ],1)
        self.λ = unsqueeze(λ.max(1)[0], (1,2,3))

        shuffle = torch.randperm(self.yb.size(0))
        xb1, self.yb1 = self.xb[shuffle], self.yb[shuffle]
        self.learner.xb = lin_comb(self.λ, self.xb, xb1)

    def after_fit(self):
        self.learner.loss_func = self.old_loss_func

    def loss_func(self, pred, yb):
        if not self.in_train: return self.old_loss_func(pred, yb)
        with NoneReduce(self.old_loss_func) as loss_func:
            loss = loss_func(pred, yb)
            loss1 = loss_func(pred, self.yb1)
        loss = lin_comb(self.λ, loss, loss1)
        return reduce_loss(loss, getattr(self.old_loss_func, 'reduction', 'mean'))

# Hooks

In [ ]:
class Hook():
    def __init__(self, m, f): self.hook = m.register_forward_hook(partial(f, self))
    def remove(self): self.hook.remove()
    def __del__(self): self.remove()

def append_stats(hook, mod, inp, out):
    if not hasattr(hook, 'stats'): hook.stats=([],[])
    means, stds = hook.stats
    means.append(out.data.mean().cpu())
    stds.append(out.data.std().cpu())

class Hooks(ListContainer):
    def __init__(self, ms, f): super().__init__([Hook(m,f) for m in ms])
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.remove()
    def __del__(self): self.remove()

    def __delitem__(self, i):
        self[i].remove()
        super().__delitem__(i)

    def remove(self):
        for h in self: h.remove()

def hook_outputs(hook, mod, inp, outp):
    hook.output = outp

# Get_batch, Model_summary, is_lin_layer

In [ ]:
def get_batch(dl, learner):
    learner.xb, learner.yb = next(iter(dl))
    learner.do_before_fit(0)
    learner('before_batch')
    learner('after_fit')
    return learner.xb, learner.yb

def is_lin_layer(l):
    lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
    return isinstance(l, lin_layers)

def find_modules(m, cond):
    if cond(m): return [m]
    return sum([find_modules(o, cond) for o in m.children()],[])


def model_summary(learner, find_all=False, print_mod = False):
    xb, yb = get_batch(learner.data.valid_dl, learner)
    mods = find_modules(learner.model, is_lin_layer) if find_all else learner.model.children()
    f = lambda hook, mod, inp, outp: print(f"========\n{mod}\n" if print_mod else "", outp.shape)
    with Hooks(mods, f) as hooks: learner.model(xb)

#cnn_learner

In [ ]:
def cnn_learner(model_arch, data, loss_func, opt_func, c_in=None, c_out=None, lr = 1e-2, cuda=True, norm = None, progress = True, mixup = 0, extra_cbs=None, **kwargs):
    cbfs = [partial(AvgStatsCallback, accuracy)] + listify(extra_cbs)
    if progress: cbfs.append(ProgressbarCallback)
    if cuda    : cbfs.append(CudaCallback)
    if norm    : cbfs.append(partial(BatchTransformXCallback, norm))
    if mixup   : cbfs.append(partial(MixUp, mixup))
    arch_args={}
    if not c_in: c_in=data.c_in
    if not c_out: c_out=data.c_out
    if c_in:arch_args['c_in'] = c_in
    if c_out:arch_args['c_out'] = c_out
    print(arch_args)
    return Learner(model_arch(**arch_args), data, loss_func, opt_func = opt_func, lr=lr, cbfuncs=cbfs, **kwargs)



# Save and Load Model

In [ ]:
class SaveModelCallback(Callback):
    _order=100
    def __init__(self, model_path, interval=2):
        self.model_path = model_path
        self.interval=interval

        if not os.path.exists(self.model_path):
            os.mkdir(self.model_path) 

    def after_epoch(self):
        if (self.learner.epoch+1) % self.interval==0:
            state = {
                'epoch':self.learner.epoch,
                'model':self.learner.model.state_dict(),
                'optimizer':self.learner.opt.state_dict()
            }
            
            torch.save(state,f'{self.model_path}/{self.epoch}.pth')

class LoadModelCallback(Callback):
    _order = -50
    def __init__(self, model_path, from_start=False, with_opt=True):
        self.model_path = model_path
        self.with_opt = with_opt
        self.from_start = from_start
        if not os.path.exists(self.model_path):
            print('Invalid model Path')
            return
        self.state = torch.load(self.model_path)

    def before_fit(self):
        self.learner.model.load_state_dict(self.state['model'])
        if self.with_opt:self.learner.opt.load_state_dict(self.state['optimizer']) 
    
    def before_epoch(self):
        if self.from_start:return
        self.learner.epoch = self.state['epoch'] +self.learner.epoch +1
        if self.learner.epoch == self.learner.epochs:raise CancelTrainException()

# Display

In [ ]:
def show_results(learn, num_of_outputs=5, valid=False,vmin=1,vmax=31):
    columns=3
    total = num_of_outputs*columns
    rows = num_of_outputs
    fig,axes = plt.subplots(rows,columns,figsize=(columns*10,rows*10))
    i=0
    dl = learn.data.valid_dl if valid else learn.data.train_dl
    for xb,yb in dl:
        for x,y in zip(xb,yb):
            show_image(x, axes.flat[i],figsize=(32,32),title='Input')
            i+=1
            show_image(y, axes.flat[i],figsize=(32,32),title='Target',vmin=vmin, vmax=vmax)
            i+=1
            
            show_image(learn.model.cpu()(x.unsqueeze(0)).argmax(dim=1).squeeze(0), axes.flat[i],figsize=(32,32),title='Prediction',vmin=vmin,vmax=vmax)
            i+=1

            if i==total:return

In [ ]:
class ShowImgsCallback(Callback):
    #displays images after each epoch
    def __init__(self, show_img_interval=1):
        self.show_img_interval=show_img_interval
        assert show_img_interval, "Non_zero allowed"

    def before_fit(self):
        self.imgs = []
        self.titles = []
        assert hasattr(self.learner,'progressbar')

    def after_epoch(self):
        if (self.learner.epoch+1) % self.show_img_interval == 0:
    
            self.last_gen = torch.cat((self.learner.xb[1].detach()/2 +0.5,self.learner.yb[1].detach()/2 + 0.5 ,self.learner.pred[1].detach()/2 +0.5),dim=-1)
            self.imgs.append(self.last_gen)
            self.titles.append(f'Epoch {self.learner.epoch}\n Input/Target/Prediction')
            self.progressbar.mbar.show_imgs(self.imgs, self.titles, imgsize=10)

In [8]:
#hide
!pip install fire
!python dl_lib/notebook2script.py notebooks/core/callbacks.ipynb dl_lib/core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Converted notebooks/core/callbacks.ipynb to dl_lib/core/callbacks.py
